<a href="https://colab.research.google.com/github/sophiamaria05/IC_MDA/blob/main/xml_to_plantuml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparing

In [55]:
#@title Defines break and tab for writing the puml file
_break = """
"""
_tab = '  '

##Importing libraries

In [56]:
#reading xml files
import xml.etree.cElementTree as ET
#importing files
import os

##Defining methods

In [57]:
#@title ***get_xml_files(xml_file_folder):*** Geting the xml files (of the predicted phrases)
def get_xml_files(xml_file_folder):
  """
    Gets the folder path where the xml file with predicted phrases.
    Returns a List[xml_files].
  """
  return [file for file in os.listdir(xml_file_folder) if file.endswith('.xml')]

In [58]:
#@title ***tag_to_array(tag):*** Converts phrase's ***type*** to array to make checking each part of the tag easier
def tag_to_array(tag):
  tag_array = [int(tag_part) for tag_part in tag.split(".")]#gets each part of the phrase's type and saves it into tag_array
  while len(tag_array) < 3:#if the type is 0 or 0.1 it appends 0s to the tag for the type of noun and reserved verb check
    tag_array.append(0)
  return tag_array#returs the array of the particioned type tag

In [59]:
#@title ***get_classes_and_associations(root_xml):*** Creates two dictionaries with the classes and it's methods, attributes and associations.
def get_classes_and_associations(root_xml):
  """
    get_classes_and_associations(root_xml) -> [methods_and_attributes, associations]

    Creates two dictionaries of the classes, one with it's methods and attributes, and the other with it's associations.

    Args:
      root_xml: The predicted xml root.

    Returns:
      A Tuple of two dictionaries.

        methods_and_attributes: {class: [methods, attributes]}
          A dictionary with the classes and it's methods and attributes.

        associations: {class: [classes_associated]}
          A dictionary with the classes and it's associations.
  """
  methods_and_attributes = {}#defines the dictionaries for the methods and attributes of each class
  associations = {}#defines the dictionaries for the associations bettween classes
  for use_case in root_xml:
    for flow in use_case:#goes through each use case in the xml file
      use_case_name = "_".join(use_case.text.split(" "))
      inputs = []#List of inputs for the "Insere" case
      for phrase in flow:#goes through each phrase of the main flow and the alternatives flows
        nodes = [node.text for node in phrase.iter()]
        while None in nodes:#removes all Nones from the phrase nodes
          nodes.pop(nodes.index(None))
        if phrase.text in nodes:#if text of the phrase is in the nodes array
          nodes.pop(nodes.index(phrase.text))#pops the phrase.text
        try:
          class_name = nodes[-1].text
        except Exception as e:
          class_name = nodes[-1]
        print("class_name: ", class_name)
        tag = tag_to_array(phrase.get("type"))
        if tag[0]==0:#
          inputs = []
          continue
        if tag[2]==1:#special case (informa and insere)
          if (phrase.find("verb").text=="Informar" or phrase.find("verb").text=="Informa") and tag[1]==1:
            nodes = [node.text for node in phrase.iter()]
            method_name = "Informa(msg)"
            try:
              methods_and_attributes["V_"+use_case_name+"  <<view>>"].append(method_name)
            except:
              methods_and_attributes["V_"+use_case_name+"  <<view>>"]=[method_name]

          elif (phrase.find("verb").text=="Inserir" or phrase.find("verb").text=="Insere"):# and tag[1]==1:#      INSERE TEST *****
            inputs = nodes[2:]

            attributes = inputs
            for attribute_name in attributes:
              try:
                methods_and_attributes[class_name].append(attribute_name)
              except:
                methods_and_attributes[class_name]=[attribute_name]

        elif tag[1]==0:
          method_name = ("_".join([phrase.find("verb").text, phrase.find("object").text]))

          if inputs!=[]:#                                                                                         INSERE TEST *****
            method_name+=("("+",".join(inputs))+")"
            inputs = []
          else:
            method_name+="()"

          try:
            methods_and_attributes["V_"+use_case_name+"  <<view>>"].append(method_name)
          except:
            methods_and_attributes["V_"+use_case_name+"  <<view>>"]=[method_name]
          try:
            methods_and_attributes["C_"+use_case_name+"  <<controller>>"].append(method_name)
          except:
            methods_and_attributes["C_"+use_case_name+"  <<controller>>"]=[method_name]

          try:#                                                                                                 ASSOCIATION TEST *****
            associations["C_"+use_case_name].append("V_"+use_case_name)
          except:
            associations["C_"+use_case_name]=["V_"+use_case_name]
          if not (class_name in associations["C_"+use_case_name]):
            associations["C_"+use_case_name].append(class_name)

        else:
          method_name = ("_".join([phrase.find("verb").text, phrase.find("object").text]))

          if inputs!=[]:#                                                                                       INSERE TEST *****
            method_name+=("("+",".join(inputs))+")"
            inputs = []
          else:
            method_name+="()"

          try:
            methods_and_attributes[class_name].append(method_name)
          except:
            methods_and_attributes[class_name]=[method_name]
  return methods_and_attributes, associations

In [60]:
#@title ***write_class_diagram(classes_and_methods):*** Writes the puml file using the classes dictionary created
def write_class_diagram(classes_and_methods, associations):
  plantuml = """@startuml classDiagram

  <style>
      classDiagram{
          class{
              .view{
                  BackgroundColor lightgreen
              }
          }
      }
  </style>
  hide <<view>> stereotype
  """

  for class_name in classes_and_methods:
    plantuml += _break+"class "+class_name+"{"
    for method in classes_and_methods[class_name]:
      plantuml += _break+_tab+method
    plantuml += _break+"}"+_break
  plantuml += _break
  for association in associations:#                                                                             ASSOCIATION TEST *****
    for class_name in associations[association]:
      plantuml += _break+association+" -- "+class_name+_break
  plantuml += _break+"@enduml"
  return plantuml

#Main method

In [61]:
#@title ***get_pumls(xml_file_folder)***
def get_pumls(xml_file_folder):
  pumls = []
  i=0
  for xml_file in get_xml_files(xml_file_folder):
    print("xml_file: ", xml_file)
    root_xml = ET.parse(xml_file).getroot()
    print("root_xml: ", root_xml)
    classes_and_methods, associations = get_classes_and_associations(root_xml)
    print("classes_and_methods: ", classes_and_methods)
    puml = write_class_diagram(classes_and_methods, associations)
    print("\n")
    if i==0:
      puml_name = 'class_diagram.puml'
    else:
      puml_name = 'class_diagram('+ str(i) +').puml'
    i+=1
    with open(puml_name, 'w') as file:
      file.write(puml)
    pumls.append(puml)
  return pumls

#Writing plantumls

In [62]:
#@title Folder where the **xml files** of the phrases was uploaded
xml_file_folder = '/content'

In [63]:
#@title Genrerate and print the pumls
i = 0
for puml in get_pumls(xml_file_folder):
  #print("Puml "+str(i)+":")
  i+=1
  print(puml)
  print("\n\n\n")

xml_file:  predicted_phrases_types.xml
root_xml:  <Element 'root' at 0x79a5a33c91c0>
[None, 'Funcionário', 'Inserir', 'Usuário']
None in nodes True
phrase.text in nodes True
phrase.text == None True
['Funcionário', 'Inserir', 'Usuário']
class_name:  Usuário
[None, 'Funcionário', 'Enviar', 'Acesso']
None in nodes True
phrase.text in nodes True
phrase.text == None True
['Funcionário', 'Enviar', 'Acesso']
class_name:  Acesso
[None, 'Sistema', 'Validar', 'Acesso']
None in nodes True
phrase.text in nodes True
phrase.text == None True
['Sistema', 'Validar', 'Acesso']
class_name:  Acesso
[None, 'Administrador', 'Inserir', 'Nome', 'Produto']
None in nodes True
phrase.text in nodes True
phrase.text == None True
['Administrador', 'Inserir', 'Nome', 'Produto']
class_name:  Produto
[None, 'Administrador', 'Enviar', 'Produto']
None in nodes True
phrase.text in nodes True
phrase.text == None True
['Administrador', 'Enviar', 'Produto']
class_name:  Produto
[None, 'Sistema', 'Validar', 'Produto']
None